### INST326 OOP Project 03

Rename this notebook, replacing "_Assignment" with "_YourName"<br>
Insert Signature Block Here

#### Group 54: Christian Sorensen, Tamunosaki Danagogo, Marcos Alvarado
> INST326
> Project 3
> 11/1/2024
#### Honor Pledge
> I pledge that the work contained in this assignment is my own, and that I have complied with University and course policies on academic integrity and AI use.


### The Project
Everyone will do the same project this time. This is a group project, so you must work in your assigned groups. Include the link to your group's GitHub repository (one link per group). Use comments in your code to document your solution. If you need to write comments to the grader, add a markdown cell immediately above your code solution and add your comments there. Be sure to read and follow all the requirements and the Notebook Instructions at the bottom of this notebook. Your grade may depend on it!

#### 1. A Scheduling Program
>  My wife is responsible for scheduling caregivers for her 93 year-old mother. Currently she writes out the schedule on a monthly calendar and photocopies it for everyone. I want all of you to help me write a program to help her with scheduling. While this is a specific application, this program will be broadly useful and adaptable to any scheduling needs for small businesses, clubs, and more.

#### Requirements
>  Care is required 12 hours per day, 7 days a week. There are two shifts each day: 7:00 AM - 1:00 PM, and 1:00 PM to 7:00 PM. There are a total of 8 caregivers. Some are family members and some are paid. Each caregiver has their own availability for shifts that is generally the same from month to month, but there are exceptions for work, vacations, and other responsibilities. Your program should do the following:
> 1. Manage caregivers and their schedules. Attributes include: name, phone, email, pay rate, and hours.
> 2. Each caregiver should have their own availability schedule where they can indicate their availability for each shift. Availability categories are 'preferred', 'available' (default), and 'unavailable'.
> 3. Create a care schedule that covers AM and PM shifts and displays caregiver names on a calendar (see example). The schedule should accomodate caregivers' individual schedules and availability preferences. The python calendar module provides options for creating HTML calendars. Sample code for the HTML calendar is in the project folder.
> 4. Paid caregivers are paid weekly at $20/hr. Your program should calculate weekly pay based on assigned hours. Provide a separate pay report that lists weekly (gross: hours x rate) amounts to each caregiver, along with weekly and monthly totals. The report can be a text document, or presented in GUI or HTML format. 

#### Group Requirements
>  1. Your submitted project should follow OOP principles like abstraction, encapsulation, inheritance, and polymorphism as appropriate. Your program should use classes. 
>  2. Select a group leader who will host the group's project repository on their GitHub.
>  3. Create the group repository and add a main program document. See example.
>  4. Create branches off the main program for each group member, and assign part of the program to each member.
>  5. Each member should work on their branch.
>  6. When each member is finished, merge the branches back into the main program. You may use 'merge' or 'pull requests', your choice.
>  7. iterate and debug as necessary.

#### Working with HTML
> Since this is a course on python, not HTML, you are not expected to know HTML. Therefore, you may copy applicable portions of the sample code or use AI to write the HTML portions of your application. Ypu should write the main python code yourself.


#### What you need to turn in
>  This is a group project. There will be one submission per group. Your submission will be graded as a group.
>  1. Include your group number and the names of all group members in the signature block at the top of this notebook.
>  2. In the cell below, paste the link to your project repository. One link per group. The grader will review the activity and history provided by GitHub. To add a hyperlink to a Jupyter markdown cell, follow the instructions in the cell below.
>  3. Below the GitHub Repository Link cell is a code cell. Copy and paste your final program code into this cell.

#### GitHub Repository Link
> [Github Repo](https://github.com/chrissor-umd/INST326_Group_54)

In [ ]:
# Solution - enter your code solution below
import tkinter as tk
import datetime as dt # For handling data and time calculations
from datetime import time
import json # For saving and loading data

# Stuff used for the calendar
import calendar
from IPython.display import HTML


class ScheduleManager():
    def __init__(self, day_start="00:00", day_end="23:59", shift_length_minutes=30):
        self.day_start = day_start # Earliest possible shift
        self.day_end = day_end # No more shifts will be generated past this time
        self.shift_length = shift_length_minutes # How long a shift is in minutes. When new shifts are added, the given time interval is broken down into shifts of this many minutes, which are each added to the schedule
        self.__load_json() # Loads necessary data for managing people and schedules


    #############################################################################################
    #######    SUPER AWESOME AND NOT ANNOYING JSON ENCODING STUFF I SPENT 6 HOURS ON :)    ######
    #############################################################################################

    @staticmethod
    def __json_people_hook(obj): # Converts JSON object into Person object when loading
        if "name" in obj:
            dict = obj
            name = dict["name"]
            #id = int(dict["_id"])
            phone = dict["phone"]
            email = dict["email"]
            pay_rate = dict["_Person__pay_rate"]
            shift_availability = dict["_Person__shift_availability"]
            shift_exceptions = dict["_Person__shift_exceptions"]
            person = Person(name, phone, email)
            person._Person__pay_rate = pay_rate
            person._Person__shift_availability = shift_availability
            person._Person__shift_exceptions = shift_exceptions
            return person
        return obj
    
    @staticmethod
    def __encode_person(obj):
        if isinstance(obj, Person):
            return obj.__dict__ # Converts a person into a serializable dictionary
        else:
            return super().default(obj)

    def __load_json(self):
        """Loads stored data from json files"""
        try:
            with open('people_data.json', 'r') as people_file:
                people_data = people_file.read()
                if len(people_data) > 0:
                    self.__people = json.loads(people_data, object_hook=self.__json_people_hook) # For some reason, I can only get this to work right by storing and loading it as a string like this
                else:
                    self.__people = {}
                print("Loaded people_data.json")
        except FileNotFoundError:
            with open('people_data.json', 'w') as people_file:
                self.__people = {}
                json.dump(self.__people, people_file, default=self.__encode_person)
                print("Created people_data.json")
        try:
            with open('shifts_data.json', 'r') as shifts_file:
                shifts = shifts_file.read()
                if len(shifts) > 0:
                    self.__shifts = json.loads(shifts)
                    print("Loaded shifts_data.json")
                else:
                    self.__shifts = self.generate_empty_shifts(day_start=self.day_start, day_end=self.day_end, shift_length=self.shift_length)
        except FileNotFoundError:
            with open('shifts_data.json', 'w') as shifts_file:
                self.__create_shifts()
                json.dump(self.__shifts, shifts_file)
                print("Created shifts_data.json")

    def __save_json(self):
        """Handles data conversions for smooth json serialization, and loads data to file"""
        with open('people_data.json', 'w') as people_file:
            json_dump = json.dumps(self.__people, indent=4, default=self.__encode_person)
            people_file.write(json_dump)
        with open('shifts_data.json', 'w') as shifts_file:
            json_dump = json.dumps(self.__shifts, indent=4)
            shifts_file.write(json_dump)
            
    ################################################
    #######         END OF JSON STUFF         ######
    ################################################


    def add_person(self, name, phone=None, email=None):
        """Creates a new person object with a unique id"""
        person_id = name.upper()
        person = Person(name, phone, email) # Creates the person object
        if person_id in self.__people.keys():
            print("Person already exists")
            return person
        person.create_availability_dict(self.shift_length)
        self.__people.update({person_id:person}) # Stores the new person in the people dictionary, with their name as the key
        print(f"Added {person.name}")
        self.__save_json()
        return person
    
    def get_person(self, name): # Get the person object associated with a given name
        if name.upper() in self.__people:
            return self.__people[name.upper()]
        else:
            print("Person not found")

    def get_person_id(self, person_or_name): # Gets a formatted person_id from either their name or a person object
        if isinstance(person_or_name, Person):
            person_id = person_or_name.name.upper()
        elif isinstance(person_or_name, str):
            person_id = person_or_name.upper()
            if person_id not in self.__people.keys():
                raise Exception("Person does not exist")
        return person_id

    def remove_person(self, person_id):
        """Removes person from ScheduleManager"""
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        for shifts_day in self.__shifts.values(): # Get each day's list of shifts
            if person_id in shifts_day: # Remove the person's shifts from each day
                shifts_day.pop(person_id)
        self.__people.pop(person_id) # Removes person from list
        print(f"{person.name} removed")
        self.__save_json()
    
    @staticmethod
    def generate_empty_shifts(day_start="00:00", day_end="23:59", shift_length=30):
        """Returns dictionary with empty shifts for each day"""
        _, day_start, day_end = ScheduleManager.get_formatted_shift_input('sun', day_start, day_end) # Only the formatted times are needed, so sunday is a placeholder
        null_date = dt.datetime(2000, 1, 1) # Null date just so the addition works
        current_shift = dt.datetime.combine(null_date, day_start)
        end = dt.datetime.combine(null_date, day_end)
        delta = dt.timedelta(minutes=shift_length)
        shift_time_list = []
        while current_shift < end and current_shift.day == 1:
            hour = current_shift.hour
            minute = current_shift.minute
            shift_time = time(hour=hour, minute=minute).isoformat('minutes')
            shift_time_list.append(shift_time)
            current_shift += delta
        days = {'SUN','MON','TUE','WED','THU','FRI','SAT'} # Some of this is probably redundant but I don't want any weird indexing issues
        shifts_dict = {'SUN':{},'MON':{},'TUE':{},'WED':{},'THU':{},'FRI':{},'SAT':{}} # Each day is a dictionary that will contain a day and an associated dictionary of shifts, each containing a list of scheduled people
        for day in days:
            for s in shift_time_list:
                shifts_dict[day].update({s:[]})
        print("Generated empty shifts")
        return shifts_dict.copy()
        
    def __create_shifts(self):
        """Restores default empty shift dictionary"""
        self.__shifts = self.generate_empty_shifts(day_start=self.day_start, day_end=self.day_end, shift_length=self.shift_length)

    def assign_shift(self, person_id, day, start, end):
        day, time_start, time_end = self.get_formatted_shift_input(day, start, end)
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        shift_added = False
        for shift in self.__shifts[day].keys(): # Get list of shifts
            time_index = time.fromisoformat(shift) # Convert to time. format for comparisons
            if time_start <= time_index < time_end: # Check if shift is within the given time interval
                if person_id not in self.__shifts[day][shift]: # Prevent duplicate entries
                    self.__shifts[day][shift].append(person_id)
                    shift_added = True
        if shift_added:
            self.__save_json()
            print(f"Added {person.name} to shifts") # In case they're already fully booked for that time frame


    def set_availability(self, person_id, day, start, end, availability):
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        person.set_availability(day, start, end, availability)
        self.__save_json()

    @staticmethod
    def check_overlap(interval1, interval2):
        """Checks if there is any kind of overlap between two time intervals, and returns the type of overlap as a str, or False if no overlap is present"""
        time_list = [interval1[0],interval1[1],interval2[0],interval2[1]]
        for i, t in enumerate(time_list): # Convert each value to time. format if necessary
            if isinstance(t, str):
                try:
                    converted = time.fromisoformat(t) # Try converting time to time. format
                    time_list[i] = converted # Replace original value in list
                except ValueError:
                    raise Exception("Invalid time input") # This is just so I can have a custom error message
        int1_start = time_list[0]
        int1_end = time_list[1]
        int2_start = time_list[2]
        int2_end = time_list[3]
        if int1_start > int1_end or int2_start > int2_end: # start time should always be less than end time
            raise Exception("Invalid time input")
        if int1_end < int2_start or int1_start > int2_end: # No overlap
            return False
        elif (int1_start >= int2_start and int1_end <= int2_end) or (int1_start <= int2_start and int1_end >= int2_end): # Full overlap
            return 'full'
        elif int1_start > int2_start: # Left overlap
            return 'right'
        elif int1_end < int2_end: # Right overlap
            return 'left'

    @staticmethod
    def get_formatted_shift_input(day, start, end): # This is a staticmethod so any class can call it easily
        """Returns a formatted and typechecked tuple of (day, time_start, time_end), raising error on invalid input. Day can be single day as string, or list of strings"""
        if (type(day) != str and type(day) != list) or type(start) != str or type(end) != str:
            raise Exception("Invalid format")
        if type(day) == str:
            day = day[0:3].upper() # Gets the uppercase forms of the first 3 letters of day, so inputs like 'Sunday' and 'sun' are both accepted
            if day not in {'SUN','MON','TUE','WED','THU','FRI','SAT'}:
                raise Exception("Invalid day")
        elif type(day) == list:
            for d in enumerate(day):
                index = d[0]
                current_day = d[1]
                if type(current_day) != str or current_day not in {'SUN','MON','TUE','WED','THU','FRI','SAT'}:
                    raise Exception("Invalid day format")
                day[index] = current_day[0:3].upper() # Gets the uppercase forms of the first 3 letters of day, so inputs like 'Sunday' and 'sun' are both accepted
        try:
            time_start = time.fromisoformat(start) # Ensure that the time is in a valid convertable format
            time_end = time.fromisoformat(end)
        except ValueError: # If the time format cannot be converted, try messing with the string a little for easy fixes
            try:
                time_list = []
                for t in (start, end):
                    tlist = t.split(":")
                    time_string = ""
                    for num in tlist:
                        if len(num) < 2: # Time conversion errors out if single digits are not prefixed with 0, so this tries to anticipate that
                            zeroes_to_add = 2-len(num)
                            num = ('0'*zeroes_to_add)+num # Prefixes zeroes
                        elif len(num) > 2:
                            num = num[0:2] # Truncate number to first 2 digits
                        time_string += f"{num}:" # Adds number to time_string, adding a colon
                    time_string = time_string[0:-1] # Remove last colon
                    time_list.append(time_string)
                time_start = time.fromisoformat(time_list[0]) # Retry conversion with modified inputs
                time_end = time.fromisoformat(time_list[1])
            except AttributeError or ValueError:
                raise Exception("Invalid time format")
        if time_start > time_end:
            raise Exception("Invalid time interval")
        for itime in (time_start, time_end): # Do some rounding and formatting for nicer and more predictable time intervals
            itime.replace(second=0)
            itime.replace(minute=(itime.minute - itime.minute%5)) # Round down to the nearest interval of 5 minutes
        return (day, time_start, time_end) # Returns a tuple containing the formatted and typechecked inputs

    def _debug_print_shifts(self): # DELETE THIS ONCE WE GET EVERYTHING ON THE HTML/GUI
        tab = "    "
        for day in self.__shifts:
            print(day)
            for person_id in self.__shifts[day]:
                print(f"{tab}{self.__people[person_id].name}:")
                for shift in self.__shifts[day][person_id]:
                    print(f"{tab*2}{shift[0]}-{shift[1]}")


class Person:
    def __init__(self, name, phone=None, email=None):
        self.name = str(name)
        self.phone = phone # Optional
        self.email = email # Optional
        self.__pay_rate = 0
        self.__shift_exceptions = {}

    def __str__(self):
        """Returns name of person when object is converted to string"""
        return str(self.name)
    
    @property
    def pay_rate(self):
        """Read Only"""
        return self.__pay_rate

    def set_pay_rate(self, new_rate):
        """Sets the new rate, checking the type and rejecting negative numbers"""
        if type(new_rate) in {float, int}:
            if new_rate >= 0:
                self.__pay_rate = new_rate
            else:
                print("New rate must be positive number or zero")
        else:
            print("Invalid pay rate amount")

    @property
    def availability(self):
        """Read only"""
        return self.__shift_availability

    def create_availability_dict(self, shift_length):
        start = dt.datetime(2000, 1, 1) # Null date just so the addition works
        current_shift = start
        delta = dt.timedelta(minutes=shift_length)
        shift_time_list = []
        while current_shift.day == 1:
            hour = current_shift.hour
            minute = current_shift.minute
            shift_time = time(hour=hour, minute=minute).isoformat('minutes')
            shift_time_list.append(shift_time)
            current_shift += delta
        days = {'SUN','MON','TUE','WED','THU','FRI','SAT'} # Some of this is probably redundant but I don't want any weird indexing issues
        self.__shift_availability = {'SUN':{},'MON':{},'TUE':{},'WED':{},'THU':{},'FRI':{},'SAT':{}} # Each day is a dictionary that will contain a day and an associated dictionary of shifts, each containing a list of scheduled people
        for day in days:
            for s in shift_time_list:
                self.__shift_availability[day].update({s:'available'})
        print(f"Created availability dictionary for {self.name}")

    def set_availability(self, day, start, end, availability):
        """Changes the availability for the given day and time"""
        availability = availability.lower()
        if availability not in {'preferred', 'available', 'unavailable'}: # Check for valid availability
            print("Invalid availability, please select only from: 'available', 'unavailable', 'preferred'")
            return
        day, time_start, time_end = ScheduleManager.get_formatted_shift_input(day, start, end)
        availability_dict = self.__shift_availability[day].copy()
        for shift in availability_dict.keys(): # Get list of shifts
            time_index = time.fromisoformat(shift) # Convert to time. format for comparisons
            time_string = time_index.isoformat('minutes')
            if time_start <= time_index < time_end: # Check if shift is within the given time interval
                self.__shift_availability[day].update({time_string:availability})
        print(f"Changed availability to '{availability}' for shifts from {start} to {end}")


    def get_availability(self, day, start, end):
        pass

    def add_shift_exception(self, date, start, end, availability):
        """Adds an availaibility exeption for a specific date"""
        # My intention for this is that once we implement functionality for getting the current date and time, we can compare it to what's in the exception dict to overwrite the default values for that week
        pass

import csv

class WorkSchedule:
    def __init__(self):
        self.employees = {}
        self.shifts = {}

    def add_employee(self, name: str) -> None:
        """Add a new employee to the schedule."""
        name = name.strip().title()
        if name not in self.employees:
            self.employees[name] = name.upper()
            print(f"Employee '{name}' added.")
        else:
            print(f"Employee '{name}' already exists.")

    def assign_shift(self, name: str, day: str, start_time: str, end_time: str) -> None:
        """Assign a shift to an employee on a specific day."""
        name = name.strip().title()
        if name not in self.employees:
            print(f"Error: Employee '{name}' not found.")
            return
        
        day = day.strip().lower()
        if day not in self.shifts:
            self.shifts[day] = {}
        if name.upper() not in self.shifts[day]:
            self.shifts[day][name.upper()] = []
        
        self.shifts[day][name.upper()].append((start_time, end_time))
        print(f"Shift added for '{name}' on {day.capitalize()} from {start_time} to {end_time}.")

    def calculate_hours(self, start_time: str, end_time: str) -> float:
        """Calculate hours worked between start and end times."""
        from datetime import datetime
        time_format = "%H:%M"
        start = datetime.strptime(start_time, time_format)
        end = datetime.strptime(end_time, time_format)
        delta = (end - start).seconds / 3600.0
        return delta

    def generate_pay_report(self, hourly_rate: float = 20.0) -> None:
        """Generate and save the pay report with only pay details, excluding shifts or employee data."""
        report_filename = "pay_report.csv"
        
        with open(report_filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Employee Name", "Total Hours Worked", "Weekly Pay", "Monthly Pay"])

            for name in self.employees.values():
                # Calculate total hours worked by summing hours for each shift assigned to the employee
                total_hours = 0
                for day, shifts in self.shifts.items():
                    if name.upper() in shifts:
                        for shift in shifts[name.upper()]:
                            start_time, end_time = shift
                            total_hours += self.calculate_hours(start_time, end_time)
                
                # Calculate weekly and monthly pay
                weekly_pay = total_hours * hourly_rate
                monthly_pay = weekly_pay * 4  # Assuming 4 weeks per month for simplicity
                
                # Write the pay details to the CSV report
                writer.writerow([name, total_hours, weekly_pay, monthly_pay])
        
        print(f"Pay report saved to {report_filename}.")

# Example Usage
work_schedule = WorkSchedule()
work_schedule.add_employee("Alice")
work_schedule.add_employee("Bob")

# Assign shifts
work_schedule.assign_shift("Alice", "Monday", "09:00", "17:00")
work_schedule.assign_shift("Alice", "Wednesday", "09:00", "17:00")
work_schedule.assign_shift("Bob", "Tuesday", "10:00", "18:00")

# Generate pay report (just pay-related data, no shift data)
work_schedule.generate_pay_report(hourly_rate=25.0)


#Calendar stuff
class CalendarApp:
    def __init__(self, root):
        self.calendar_manager = CalendarManager()
        self.root = root
        self.root.title("Calendar Viewer")
        self.setup_ui()

    def setup_ui(self):
        
        self.text_widget = tk.Text(self.root, wrap="none", width=100, height=30)
        self.text_widget.grid(row=0, column=0, columnspan=2)

        
        self.year_button = tk.Button(self.root, text="View Year", command=self.display_year)
        self.year_button.grid(row=1, column=0, padx=5, pady=5)

        self.month_button = tk.Button(self.root, text="View Month", command=self.display_month)
        self.month_button.grid(row=1, column=1, padx=5, pady=5)

        
        self.month_entry = tk.Entry(self.root, width=5)
        self.month_entry.grid(row=2, column=1, padx=5, pady=5)
        tk.Label(self.root, text="Enter Month (1-12):").grid(row=2, column=0, padx=5, pady=5)

    def display_year(self):
       
        self.calendar_manager.set_year_view()
        calendar_text = self.calendar_manager.render_calendar()
        self.update_text_widget(calendar_text)

    def display_month(self):
        
        month = self.month_entry.get()
        if month.isdigit() and 1 <= int(month) <= 12:
            self.calendar_manager.set_month_view(int(month))
            calendar_text = self.calendar_manager.render_calendar()
            self.update_text_widget(calendar_text)
        else:
            self.update_text_widget("Invalid month! Please enter a number between 1 and 12.")

    def update_text_widget(self, content):
        
        self.text_widget.delete("1.0", tk.END)  # Clear existing content
        self.text_widget.insert(tk.END, content)

class CalendarManager:
    def __init__(self):
        self.year = 2024  
        self.month = None  
        self.text_cal = calendar.TextCalendar()

    def set_year_view(self):
        
        self.month = None

    def set_month_view(self, month):
        """Switch to month view."""
        if 1 <= month <= 12:
            self.month = month
        else:
            raise ValueError("Month must be between 1 and 12.")

    def render_calendar(self):
        
        if self.month: 
            return self.text_cal.formatmonth(self.year, self.month)
        else:  
            return self.text_cal.formatyear(self.year)

# Run the Tkinter app
if __name__ == "__main__":
    root = tk.Tk()
    app = CalendarApp(root)
    root.mainloop()

 


calendar_manager = CalendarManager()




if __name__ == "__main__":
    schedule = ScheduleManager()
    person1 = schedule.add_person('Joe')
    person2 = schedule.add_person('Bob')
    schedule.assign_shift(person1, 'sun', '10:30','17:00')
    schedule.set_availability(person1, 'sun', '5:30', '10:30', 'unavailable')

Changed availability to 'unavailable' for shifts from 5:30 to 10:30


### Notebook Instructions
> Before turning in your notebook:
> 1. Make sure you have renamed the notebook file as instructed
> 2. Make sure you have included your signature block and that it is correct according to the instructions
> 3. comment your code as necessary
> 4. run all code cells and double check that they run correctly. If you can't get your code to run correctly and you want partial credit, add a note for the grader in a new markdown cell directly above your code solution.<br><br>
Turn in your notebook by uploading it to ELMS<br>
IF the exercises involve saved data files, put your notebook and the data file(s) in a zip folder and upload the zip folder to ELMS